In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE

In [12]:
#cargamos el dataset
df = pd.read_csv ('/Users/luiseduardogarciablanco/Desktop/nueva data cancer/2024/metadata_cleaned.csv', index_col=0)
df

target  age_approx     sex anatom_site_general  \
isic_id                                                        
ISIC_0015670       0        60.0    male     lower extremity   
ISIC_0015845       0        60.0    male           head/neck   
ISIC_0015864       0        60.0    male     posterior torso   
ISIC_0015902       0        65.0    male      anterior torso   
ISIC_0024200       0        55.0    male      anterior torso   
...              ...         ...     ...                 ...   
ISIC_9999937       0        70.0    male      anterior torso   
ISIC_9999951       0        60.0    male     posterior torso   
ISIC_9999960       0        65.0  female      anterior torso   
ISIC_9999964       0        30.0  female      anterior torso   
ISIC_9999967       0        50.0    male     lower extremity   

              clin_size_long_diam_mm tbp_tile_type   tbp_lv_A  tbp_lv_Aext  \
isic_id                                                                      
ISIC_0015670                    3.04     3D: white  20.244422    16.261975   
ISIC_0015845                    1.10     3D: white  31.712570    25.364740   
ISIC_0015864                    3.40        3D: XP  22.575830    17.128170   
ISIC_0015902                    3.22        3D: XP  14.242329    12.164757   
ISIC_0024200                    2.73     3D: white  24.725520    20.057470   
...                              ...           ...        ...          ...   
ISIC_9999937                    6.80        3D: XP  22.574335    14.944666   
ISIC_9999951                    3.11     3D: white  19.977640    16.026870   
ISIC_9999960                    2.05        3D: XP  17.332567    12.364397   
ISIC_9999964                    2.80        3D: XP  22.288570     9.564721   
ISIC_9999967                    3.30        3D: XP  16.792900    11.434730   

               tbp_lv_B  tbp_lv_Bext  ...  tbp_lv_stdL  tbp_lv_stdLExt  \
isic_id                               ...                                
ISIC_0015670  26.922447    23.954773  ...     2.036195        2.637780   
ISIC_0015845  26.331000    24.549290  ...     0.853227        3.912844   
ISIC_0015864  37.970460    33.485410  ...     1.743651        1.950777   
ISIC_0015902  21.448144    21.121356  ...     1.258541        1.573733   
ISIC_0024200  26.464900    25.710460  ...     2.085409        2.480509   
...                 ...          ...  ...          ...             ...   
ISIC_9999937  27.663259    26.767135  ...     7.054819        2.169398   
ISIC_9999951  34.158840    31.011870  ...     1.879502        2.910780   
ISIC_9999960  29.845326    26.500073  ...     1.702824        2.205272   
ISIC_9999964  28.431200    27.012250  ...     3.680175        1.957157   
ISIC_9999967  21.457220    21.214910  ...     3.068755        3.199104   

              tbp_lv_symm_2axis  tbp_lv_symm_2axis_angle    tbp_lv_x  \
isic_id                                                                
ISIC_0015670           0.590476                       85 -182.703552   
ISIC_0015845           0.285714                       55   -0.078308   
ISIC_0015864           0.361905                      105  123.649700   
ISIC_0015902           0.209581                      130 -141.024780   
ISIC_0024200           0.313433                       20  -72.315640   
...                         ...                      ...         ...   
ISIC_9999937           0.288920                      100  147.187256   
ISIC_9999951           0.460000                       25   52.349740   
ISIC_9999960           0.183099                       40   54.622246   
ISIC_9999964           0.161850                      140   -9.861557   
ISIC_9999967           0.197917                       45 -258.872300   

                 tbp_lv_y    tbp_lv_z  iddx_full  iddx_1  \
isic_id                                                    
ISIC_0015670   613.493652  -42.427948     Benign  Benign   
ISIC_0015845  1575.687000   57.174500     Benign  Benign   
ISIC_0015864  1472.010000 

In [13]:
# Eliminar las columnas especificadas
df.drop(columns=['iddx_full', 'iddx_1', 'anatom_site_general'], inplace=True)

# Verificar si las columnas fueron eliminadas
print(df.head())

              target  age_approx   sex  clin_size_long_diam_mm tbp_tile_type  \
isic_id                                                                        
ISIC_0015670       0        60.0  male                    3.04     3D: white   
ISIC_0015845       0        60.0  male                    1.10     3D: white   
ISIC_0015864       0        60.0  male                    3.40        3D: XP   
ISIC_0015902       0        65.0  male                    3.22        3D: XP   
ISIC_0024200       0        55.0  male                    2.73     3D: white   

               tbp_lv_A  tbp_lv_Aext   tbp_lv_B  tbp_lv_Bext   tbp_lv_C  ...  \
isic_id                                                                  ...   
ISIC_0015670  20.244422    16.261975  26.922447    23.954773  33.684638  ...   
ISIC_0015845  31.712570    25.364740  26.331000    24.549290  41.219030  ...   
ISIC_0015864  22.575830    17.128170  37.970460    33.485410  44.174920  ...   
ISIC_0015902  14.242329    12.164757  2

In [14]:
# Aplicar One-Hot Encoding a las columnas especificadas
df = pd.get_dummies(df, columns=['sex', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple'])

# Verificar las nuevas columnas
print(df.head())

              target  age_approx  clin_size_long_diam_mm   tbp_lv_A  \
isic_id                                                               
ISIC_0015670       0        60.0                    3.04  20.244422   
ISIC_0015845       0        60.0                    1.10  31.712570   
ISIC_0015864       0        60.0                    3.40  22.575830   
ISIC_0015902       0        65.0                    3.22  14.242329   
ISIC_0024200       0        55.0                    2.73  24.725520   

              tbp_lv_Aext   tbp_lv_B  tbp_lv_Bext   tbp_lv_C  tbp_lv_Cext  \
isic_id                                                                     
ISIC_0015670    16.261975  26.922447    23.954773  33.684638    28.953117   
ISIC_0015845    25.364740  26.331000    24.549290  41.219030    35.299260   
ISIC_0015864    17.128170  37.970460    33.485410  44.174920    37.611800   
ISIC_0015902    12.164757  21.448144    21.121356  25.746200    24.374023   
ISIC_0024200    20.057470  26.464900    

In [15]:
# 1. Preparar el dataset
# Dividir en características (X) y la variable objetivo (y)
X = df.drop('target', axis=1)
y = df['target']

# 2. Manejo del desbalance con SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 3. Dividir el dataset en conjunto de entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Escalar características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Definir los modelos a evaluar
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    #'Random Forest': RandomForestClassifier(n_estimators=100),
    #'Support Vector Machine': SVC(probability=True),
    #'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5)
}

# 5. Entrenar y evaluar los modelos
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]  # Para AUC-ROC si es aplicable
    
    print(f"Classification Report for {name}:")
    print(classification_report(y_test, y_pred, zero_division=0))
    
    # AUC-ROC Score
    roc_auc = roc_auc_score(y_test, y_proba)
    print(f"AUC-ROC Score for {name}: {roc_auc}")
    
    print("\n" + "-"*50 + "\n")

Training Logistic Regression...
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.93      0.96      0.95     76194
           1       0.96      0.93      0.94     76420

    accuracy                           0.94    152614
   macro avg       0.95      0.94      0.94    152614
weighted avg       0.95      0.94      0.94    152614

AUC-ROC Score for Logistic Regression: 0.9879649115282985

--------------------------------------------------



aqui se hace una prueba de modelos, que aunque hay unos mejores resultados con las variabls por si solas, separaremos los procesos, donde parte del equipo seguirá desarollando la parte de imagenes vs target y el resto trabaja con los metadatos para obtener un modelo que se pueda unir al modelo entrenado solo con imagenes, de esta forma poder pasar a un unico modelo, imagenes y metadatos para obtener una prediccion